In [1]:
import pandas as pd
import numpy as np
import re
import time

import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import ldamulticore

In [2]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [3]:
df_Q = pd.read_csv("Q.csv")
df_P = pd.read_csv("doc_P.csv")

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df = pd.concat([df_Q, df_P], axis=1)
df.drop(df.index[[76853]], axis=0, inplace=True)
df.reset_index(inplace = True) 

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,...,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,TB_RL_P,TBAA_RL_P,TBA_RL_P,TB_RLC_P,TBAA_RLC_P,TBA_RLC_P
0,0,59549258,1,59549692.0,NaN,2020-01-01 00:14:31,NaN,-1,48,<p>My bot is not working as planned.<br>\nFor ...,...,1.0,NaN,NaN,CC BY-SA 4.0,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...
1,1,59549310,1,NaN,NaN,2020-01-01 00:33:01,NaN,-2,1887,<pre><code>pi@raspberrypi:~/examples/lite/exam...,...,1.0,NaN,NaN,CC BY-SA 4.0,raspberri pi 4 tensorflow instal pi raspberryp...,raspberri pi 4 tensorflow instal pi raspberryp...,raspberri pi 4 tensorflow instal pi raspberryp...,raspberri pi 4 tensorflow instal instal use pip3,raspberri pi 4 tensorflow instal instal use pip3,raspberri pi 4 tensorflow instal instal use pi...
2,2,59549322,1,NaN,NaN,2020-01-01 00:35:38,NaN,11,3198,<p>Trying to import pytest on Python 3.8 fails...,...,NaN,NaN,NaN,CC BY-SA 4.0,import pytest fail attributeerror str object a...,import pytest fail attributeerror str object a...,import pytest fail attributeerror str object a...,import pytest fail attributeerror str object a...,import pytest fail attributeerror str object a...,import pytest fail attributeerror str object a...
3,3,59549341,1,59549351.0,NaN,2020-01-01 00:39:41,NaN,0,20,<p>Suppose I have the dataframe</p>\n\n<pre><c...,...,NaN,NaN,NaN,CC BY-SA 4.0,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...
4,4,59549365,1,NaN,NaN,2020-01-01 00:49:30,NaN,-1,647,<p>I'am learning python by doing little projec...,...,NaN,NaN,NaN,CC BY-SA 4.0,python schedul random delay learn python littl...,python schedul random delay learn python littl...,python schedul random delay learn python littl...,python schedul random delay learn python littl...,python schedul random delay learn python littl...,python schedul random delay learn python littl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164695,164696,62667715,1,NaN,NaN,2020-06-30 23:49:49,NaN,0,43,<p>I use keras with tensorflow backend i finin...,...,NaN,NaN,NaN,CC BY-SA 4.0,build cnn python train sstop traing give warn ...,build cnn python train sstop traing give warn ...,build cnn python train sstop traing give warn ...,build cnn python train sstop traing give warn ...,build cnn python train sstop traing give warn ...,build cnn python train sstop traing give warn ...
164696,164697,62667732,1,NaN,NaN,2020-06-30 23:53:34,NaN,0,23,"<p>In my views I do this:</p>\n<pre class=""lan...",...,NaN,NaN,NaN,CC BY-SA 4.0,revers idnom argument found 1 pattern tri view...,revers idnom argument found 1 pattern tri view...,revers idnom argument found 1 pattern tri view...,revers idnom argument found 1 pattern tri view...,revers idnom argument found 1 pattern tri view...,revers idnom argument found 1 pattern tri view...
164697,164698,62667746,1,NaN,NaN,2020-06-30 23:56:03,NaN,1,73,<p>I would like to store the IPaddress and loc...,...,NaN,NaN,NaN,CC BY-SA 4.0,store ipaddress locat django would like store ...,store ipaddress locat django would like store ...,store ipaddress locat django would like store ...,store ipaddress locat django would like store ...,store ipaddress locat django would like store ...,store ipaddress locat django would like store ...
164698,164699,62667760,1,NaN,NaN,2020-06-30 23:58:22,NaN,0,55,"<p>I am creating a chatting app, something lik...",...,NaN,NaN,NaN,CC BY-SA 4.0,automat updat field model django creat chat ap...,aut

In [6]:
df_AA = df[df["AcceptedAnswerId"].notnull()]
df_AA

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,...,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,TB_RL_P,TBAA_RL_P,TBA_RL_P,TB_RLC_P,TBAA_RLC_P,TBA_RLC_P
0,0,59549258,1,59549692.0,NaN,2020-01-01 00:14:31,NaN,-1,48,<p>My bot is not working as planned.<br>\nFor ...,...,1.0,NaN,NaN,CC BY-SA 4.0,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...,discord xp bot recogn user data json file bot ...
3,3,59549341,1,59549351.0,NaN,2020-01-01 00:39:41,NaN,0,20,<p>Suppose I have the dataframe</p>\n\n<pre><c...,...,NaN,NaN,NaN,CC BY-SA 4.0,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...,convert timedelta integ consecut time point pa...
5,5,59549374,1,59549386.0,NaN,2020-01-01 00:52:07,NaN,0,77,<p>How to append an array with a dictionary in...,...,NaN,NaN,NaN,CC BY-SA 4.0,python append array dictionari insid dictionar...,python append array dictionari insid dictionar...,python append array dictionari insid dictionar...,python append array dictionari insid dictionar...,python append array dictionari insid dictionar...,python append array dictionari insid dictionar...
6,6,59549393,1,59549474.0,NaN,2020-01-01 01:00:57,NaN,0,265,<p>I currently have the following models in my...,...,NaN,NaN,NaN,CC BY-SA 4.0,queri across multipl model singl django view c...,queri across multipl model singl django view c...,queri across multipl model singl django view c...,queri across multipl model singl django view c...,queri across multipl model singl django view c...,queri across multipl model singl django view c...
8,8,59549421,1,59549442.0,NaN,2020-01-01 01:09:13,NaN,0,781,<p>I'm not really sure why this autocorrect is...,...,1.0,NaN,NaN,CC BY-SA 4.0,autocorrect python librari throw error instead...,autocorrect python librari throw error instead...,autocorrect python librari throw error instead...,autocorrect python librari throw error instead...,autocorrect python librari throw error instead...,autocorrect python librari throw error instead...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164685,164686,62667480,1,62667562.0,NaN,2020-06-30 23:18:33,NaN,1,47,<p>I have an algorithm in a game that simplifi...,...,NaN,NaN,NaN,CC BY-SA 4.0,avoid interact statement algorithm game simpli...,avoid interact statement algorithm game simpli...,avoid interact statement algorithm game simpli...,avoid interact statement algorithm game simpli...,avoid interact statement algorithm game simpli...,avoid interact statement algorithm game simpli...
164689,164690,62667514,1,62667772.0,NaN,2020-06-30 23:22:35,NaN,2,692,"<pre><code>def play_tts(self,file_path):\n f...",...,NaN,NaN,NaN,CC BY-SA 4.0,play sound pyqt5 qtmultimedia def play tt self...,play sound pyqt5 qtmultimedia def play tt self...,play sound pyqt5 qtmultimedia def play tt self...,play sound pyqt5 qtmultimedia write code qtmul...,play sound pyqt5 qtmultimedia write code qtmul...,play sound pyqt5 qtmultimedia write code qtmul...
164691,164692,62667655,1,62756008.0,NaN,2020-06-30 23:40:34,NaN,0,34,"<p>I am using RASA NLU (open source), but I am...",...,NaN,NaN,NaN,CC BY-SA 4.0,data privaci rasa nlu use rasa nlu open sourc ...,data privaci rasa nlu use rasa nlu open sourc ...,data privaci rasa nlu use rasa nlu open sourc ...,data privaci rasa nlu use rasa nlu open sourc ...,data privaci rasa nlu use rasa nlu open sourc ...,data privaci rasa nlu use rasa nlu open sourc ...
164694,164695,62667700,1,62667891.0,NaN,2020-06-30 23:47:13,NaN,3,177,"<p>Sorry if this question is impossible, I'm s...",...,NaN,NaN,NaN,CC BY-SA 4.0,prevent

In [7]:
TB_RL = df_AA["TB_RL_P"].values.tolist()
TBAA_RL = df_AA["TBAA_RL_P"].values.tolist()
TBA_RL = df_AA["TBA_RL_P"].values.tolist()

TB_RLC = df_AA["TB_RLC_P"].values.tolist()
TBAA_RLC = df_AA["TBAA_RLC_P"].values.tolist()
TBA_RLC = df_AA["TBA_RLC_P"].values.tolist()

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Model Training

In [9]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wuyunwei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
data = TBAA_RL
start_time = time.time()

text = sent_to_words(data)
text = remove_stopwords(text)

id2word = corpora.Dictionary(text)
corpus = [id2word.doc2bow(t) for t in text]

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=40, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10000,
                                           iterations=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
print("--- %s seconds ---" % (time.time() - start_time))


/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--- 909.0579051971436 seconds ---


In [19]:
lda_model.save("TBAA_RL_W.model")

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
print('Perplexity: ', lda_model.log_perplexity(corpus))
print("--- %s seconds ---" % (time.time() - start_time))

coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)
print("--- %s seconds ---" % (time.time() - start_time))

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -8.576628493466387
--- 959.9507141113281 seconds ---
Coherence Score:  0.5821858343486931
--- 1284.1786851882935 seconds ---


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.146890  0.138793       1        1  10.478723
34     0.188549 -0.006107       2        1   6.120016
29     0.230686  0.075531       3        1   5.407345
7      0.089934  0.150560       4        1   4.589484
12    -0.143888  0.181578       5        1   3.715381
37     0.237236 -0.017678       6        1   3.570028
32    -0.078059  0.020139       7        1   3.559564
31     0.193046  0.025802       8        1   3.252363
3     -0.136246  0.147789       9        1   3.077085
11    -0.019561  0.123025      10        1   3.032992
23    -0.040350  0.075185      11        1   2.932905
24    -0.100368  0.163210      12        1   2.906187
0     -0.029169  0.125000      13        1   2.583685
8     -0.113694  0.073625      14        1   2.558975
25     0.051456  0.117240      15        1   2.528405
10    -0.036166  0.074238      16        1   2.489986
20     0.166035  0.066651      17        1   2.427379
17    -0.150024  0.149656      18        1   2.369296
4      0.075078 -0.091183      19        1   2.262676
27     0.078601 -0.057514      20        1   2.242996
22    -0.150111  0.013621      21        1   2.220931
35    -0.010455 -0.057589      22        1   2.152465
18     0.059508 -0.039959      23        1   2.114516
26    -0.024212 -0.015593      24        1   2.013405
38     0.079835 -0.048249      25        1   1.872534
13    -0.092007  0.080889      26        1   1.848088
16     0.054881 -0.100042      27        1   1.700943
1      0.072442 -0.132047      28        1   1.455211
9      0.135673 -0.034032      29        1   1.377761
36     0.071293 -0.099986      30        1   1.351428
19    -0.051586 -0.136642      31        1   1.287322
2     -0.089651  0.108252      32        1   1.272484
5     -0.006174 -0.092377      33        1   1.103368
15    -0.031981 -0.112765      34        1   1.043941
33    -0.189309 -0.169774      35        1   1.029716
30    -0.028721 -0.051299      36        1   0.870635
21    -0.066285 -0.233435      37        1   0.859938
28    -0.067779 -0.195001      38        1   0.809413
14    -0.168793 -0.070932      39        1   0.799856
39    -0.106556 -0.148578      40        1   0.710574, topic_info=         Term           Freq          Total Category  logprob  loglift
202      self  211328.000000  211328.000000  Default  30.0000  30.0000
84         df  136914.000000  136914.000000  Default  29.0000  29.0000
175      list  115122.000000  115122.000000  Default  28.0000  28.0000
24       file  130310.000000  130310.000000  Default  27.0000  27.0000
15       data  132313.000000  132313.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
986   product    1180.646826   13501.859884  Topic40  -4.5548   2.5101
4501       rc     647.406154    1069.819880  Topic40  -5.1556   4.4446
891     total     947.614585   13751.507815  Topic40  -4.7747   2.2719
678     count     831.609158   25506.996486  Topic40  -4.9052   1.5235
15       data     759.750670  132313.856590  Topic40  -4.9956  -0.2131

[2752 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1728      9  0.018436    aa
1728     27  0.006295    aa
1728     31  0.794540    aa
1728     35  0.180312    aa
996       8  0.999078   aaa
...     ...       ...   ...
954      12  0.020937  zone
954      18  0.081018  zone
954      34  0.896659  zone
1824     17  0.941792    zz
1824     34  0.057858    zz

[12936 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 35, 30, 8, 13, 38, 33, 32, 4, 12, 24, 25, 1, 9, 26, 11, 21, 18, 5, 28, 23, 36, 19, 27, 39, 14, 17, 2, 10, 37, 20, 3, 6, 16, 34, 31, 22, 29, 15, 40])

In [57]:
pyLDAvis.save_html(LDAvis_prepared, 'TBAA_RL_W.html')

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Topic Prediction

In [40]:
def topic_prediction (my_document):
    
    new_text_corpus = id2word.doc2bow(my_document.split())

    pred = lda_model.get_document_topics(new_text_corpus, minimum_probability=0)
    probability = 0
    for p in pred:
        if p[1] >  probability:
            topic = p[0]
            probability = p[1]
    return topic + 1
        

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
prediction = []
for i in range(len(df_P)):
    if i %10000 == 9999:
        print(i)
    if i == 76853:
        prediction.append(np.nan)
        continue
    my_document = df_P["TBAA_RL_P"][i]
    prediction.append(topic_prediction (my_document))
df_pred = pd.DataFrame(prediction, columns=['TBAA_RL_Pred'])
df_pred

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


9999
19999
29999
39999
49999
59999
69999
79999
89999
99999
109999
119999
129999
139999
149999
159999


,TBAA_RL_Pred
0,24.0
1,4.0
2,13.0
3,35.0
4,12.0
...,...
164696,28.0
164697,25.0
164698,11.0
164699,11.0


In [56]:
df_pred.to_csv("TBAA_RL_Pred.csv", index = False)

/Users/wuyunwei/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
